In [2]:
import numpy as np

In [27]:
r = np.random.randint(3, size=(5,20))

In [31]:
r

array([[1, 0, 0, 0, 0, 1, 2, 2, 1, 2, 0, 2, 0, 1, 2, 1, 2, 0, 1, 0],
       [2, 2, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0],
       [2, 0, 0, 1, 1, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 1, 2, 0, 1, 2],
       [0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 1, 1, 2, 0, 2, 0, 1, 0, 2, 0],
       [1, 2, 1, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 2, 2, 1, 0, 2, 1]])

In [32]:
(r.T * np.arange(5)).T

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 2, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0],
       [4, 0, 0, 2, 2, 4, 0, 4, 0, 0, 4, 4, 2, 2, 2, 2, 4, 0, 2, 4],
       [0, 0, 0, 3, 0, 0, 0, 3, 6, 6, 3, 3, 6, 0, 6, 0, 3, 0, 6, 0],
       [4, 8, 4, 8, 4, 0, 8, 4, 4, 0, 0, 0, 8, 4, 8, 8, 4, 0, 8, 4]])

In [8]:
np.sum(a, axis=1)

array([22, 21, 18, 10, 22])

In [41]:
X = np.random.randint(3, size=(5,20))

Y = X.T
n, d = Y.shape

a = n / np.sum(Y>0, axis=0)
Y = Y * np.log(a)
Y = Y * np.sum(Y**2, axis=0)**(-0.5)

In [46]:
Y.T.shape

(5, 20)

In [6]:
a = np.arange(10*5).reshape(10,5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [12]:
a[:,3].shape = np.arange(10)

(10,)

In [11]:
a

array([[ 0,  1,  2,  0,  4],
       [ 5,  6,  7,  1,  9],
       [10, 11, 12,  2, 14],
       [15, 16, 17,  3, 19],
       [20, 21, 22,  4, 24],
       [ 0,  1,  2,  5,  4],
       [30, 31, 32,  6, 34],
       [35, 36, 37,  7, 39],
       [40, 41, 42,  8, 44],
       [45, 46, 47,  9, 49]])